# FOR RAG with mulitple sources, we'll use tools and Agents


In [ ]:
! pip install arxiv
! pip install wikipedia

## first tool -builtin


In [34]:
# creating a wrapper

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import (
    WikipediaAPIWrapper,
)  # wikipedia already has a wrapper

wiki_api = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=200)
wiki_api

WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/zohaib/anaconda3/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)

In [35]:
wiki = WikipediaQueryRun(
    api_wrapper=wiki_api,
)
wiki.name, wiki

('wikipedia',
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/zohaib/anaconda3/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)))

## creating second tool - from scratch


In [36]:
#  reading from web
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = WebBaseLoader(web_path="https://docs.smith.langchain.com/")
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = text_splitter.split_documents(docs)
documents

[Document(page_content='Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'Introduction', 'language': 'en'}),
 Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmithâ€‹We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API keyâ€‹To create an API key head to the setting pages. Then cl

In [37]:
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e7b08df4910>)

In [38]:
#  create_retrieval_tool: Create a tool to do retrieval of documents.
# Tool class to pass to an agent

from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(
    retriever=retriever,
    name="langchain-retriever",
    description="Search for info about langsmith",
)

retrieval_tool

Tool(name='langchain-retriever', description='Search for info about langsmith', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7e7b0942d760>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e7b08df4910>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7e7b0942dc60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e7b08df4910>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

## third tool


In [39]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [40]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=100)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

### lets combine these tools


In [41]:
tools = [wiki, arxiv, retrieval_tool]

[i.name for i in tools]

['wikipedia', 'arxiv', 'langchain-retriever']

In [42]:
from dotenv import load_dotenv
import os

# env variables
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [45]:
# initializing llm

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7e7b08ae4250>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e7b08ae7410>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

### lets fetch a prompt from hub


In [50]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [51]:
# let's query these tools, here we use agents
# Agents are to use LM to choose a sequence of actions to take
# In chains sequence of actions is hardcoded
# so our agent will first look in wiki, if unsuccessful, then look in arxiv


from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', opt

## to use these agents, we need agent executer


In [57]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful

In [58]:
agent_executor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langchain-retriever` with `{'query': 'Langsmith'}`


Getting started with LangSmith | ğŸ¦œï¸�ğŸ› ï¸� LangSmith

learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Pricing: Learn about the pricing model for LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of LangSmith.Prompt Hub Learn about the Prompt Hub, a prompt management tool built into LangSmith.Additional Resourcesâ€‹LangSmith Cookbook: A collection of tutorials and end-to-end walkthroughs using LangSmith.LangChain Python: Docs for the Python LangChain library.LangChain Python API Reference: documentation to review the core APIs of LangChain.LangChain JS: Docs for the TypeScript LangChain libraryDiscord: Join us on 

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate applications, enabling quick and confident deployment. LangSmith offers Python and TypeScript SDKs for various needs. Users can create API keys, set up their environment, and log traces using LangSmith. For more information, you can refer to the LangSmith API Docs and explore the features like Tracing, Evaluation, Production Monitoring & Automations, Prompt Hub, Proxy, Pricing, and Self-Hosting. Additionally, there are resources like the LangSmith Cookbook, LangChain Python and JS documentation, and a Discord community for discussions.'}

In [59]:
agent_executor.invoke({"input": "Tell me about Allergy"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Allergy`


Page: Allergy
Summary: Allergies, also known as allergic diseases, are various conditions caused by hypersensitivity of the immune system to typically harmless substances in the environment. These disAllergies, also known as allergic diseases, are various conditions caused by hypersensitivity of the immune system to typically harmless substances in the environment. These substances that trigger allergic reactions are called allergens. Allergies can manifest in different ways, such as allergic rhinitis (hay fever), asthma, atopic dermatitis (eczema), food allergies, and more. Symptoms of allergies can range from mild to severe and can include sneezing, itching, hives, swelling, and in severe cases, anaphylaxis. Treatment for allergies may include avoiding allergens, medications, and in some cases, immunotherapy.

> Finished chain.


{'input': 'Tell me about Allergy',
 'output': 'Allergies, also known as allergic diseases, are various conditions caused by hypersensitivity of the immune system to typically harmless substances in the environment. These substances that trigger allergic reactions are called allergens. Allergies can manifest in different ways, such as allergic rhinitis (hay fever), asthma, atopic dermatitis (eczema), food allergies, and more. Symptoms of allergies can range from mild to severe and can include sneezing, itching, hives, swelling, and in severe cases, anaphylaxis. Treatment for allergies may include avoiding allergens, medications, and in some cases, immunotherapy.'}

In [60]:
agent_executor.invoke({"input": "What;s the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, TobiThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" authored by Caprice Stanley and Tobi. It discusses the concept of heat-bath random walks with Markov bases.

> Finished chain.


{'input': 'What;s the paper 1605.08386 about?',
 'output': 'The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" authored by Caprice Stanley and Tobi. It discusses the concept of heat-bath random walks with Markov bases.'}